In [ ]:
import cv2
import mediapipe as mp
import time
import math

In [ ]:
# Webcam setup
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise IOError("Cannot open webcam")
cap.set(3, 1280)
cap.set(4, 720)

# Mediapipe setup
mp_hands = mp.solutions.hands
hands_detector = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.5)
mp_draw = mp.solutions.drawing_utils

In [ ]:
class Button:
    def __init__(self, x, y, w, h, val):
        self.x, self.y, self.w, self.h = x, y, w, h
        self.val = val
        self.hovered = False

    def draw(self, img):
    # Background colors
        if self.val in ['+', '-', '*', '/', '(', ')']:
            fill_color = (255, 245, 235)         # warm cream
            border_color = (180, 200, 255)       # soft blue
        elif self.val in ['DEL', 'C']:
            fill_color = (235, 235, 255)         # light lavender
            border_color = (200, 180, 240)       # lavender border
        elif self.val == '=':
            fill_color = (220, 255, 220)         # soft green
            border_color = (140, 200, 140)       # green border
        else:
            fill_color = (245, 245, 245)         # default light gray
            border_color = (180, 180, 180)       # soft gray

        # Hover highlight
        if self.hovered:
            fill_color = tuple(min(255, c + 20) for c in fill_color)

        # Shadow offset
        shadow_offset = 2
        shadow_color = (160, 160, 160)
        cv2.rectangle(img,
                    (self.x + shadow_offset, self.y + shadow_offset),
                    (self.x + self.w + shadow_offset, self.y + self.h + shadow_offset),
                    shadow_color, -1)

        # Rounded rectangle button fill
        radius = 12
        overlay = img.copy()
        cv2.rectangle(overlay, (self.x + radius, self.y), (self.x + self.w - radius, self.y + self.h), fill_color, -1)
        cv2.rectangle(overlay, (self.x, self.y + radius), (self.x + self.w, self.y + self.h - radius), fill_color, -1)
        cv2.circle(overlay, (self.x + radius, self.y + radius), radius, fill_color, -1)
        cv2.circle(overlay, (self.x + self.w - radius, self.y + radius), radius, fill_color, -1)
        cv2.circle(overlay, (self.x + radius, self.y + self.h - radius), radius, fill_color, -1)
        cv2.circle(overlay, (self.x + self.w - radius, self.y + self.h - radius), radius, fill_color, -1)

        # Blend overlay into image
        alpha = 0.95
        cv2.addWeighted(overlay, alpha, img, 1 - alpha, 0, img)

        # Border
        cv2.rectangle(img, (self.x, self.y), (self.x + self.w, self.y + self.h), border_color, 2)

        # Text
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 1.1 if len(self.val) == 1 else 0.7
        thickness = 2
        text_size = cv2.getTextSize(self.val, font, font_scale, thickness)[0]
        text_x = self.x + (self.w - text_size[0]) // 2
        text_y = self.y + (self.h + text_size[1]) // 2 - 3
        cv2.putText(img, self.val, (text_x, text_y), font, font_scale, (0, 0, 0), thickness)
    

    def check_hover(self, cx, cy):
        self.hovered = self.x < cx < self.x + self.w and self.y < cy < self.y + self.h

    def click(self, cx, cy):
        if self.x < cx < self.x + self.w and self.y < cy < self.y + self.h:
            return True
        return False


In [ ]:
# Calculator buttons and grid
buttons = []
button_vals = [
    ['7', '8', '9', '/'],
    ['4', '5', '6', '*'],
    ['1', '2', '3', '-'],
    ['.', '0', '=', '+'],
    ['C', 'DEL', '%', '^']
]
for i, row in enumerate(button_vals):
    for j, val in enumerate(row):
        buttons.append(Button(80*j+100, 80*i+150, 75, 75, val))

# Equation and timing variables
myEquation = ''
displayEquation = ''
delayCounter = 0


In [ ]:
# ========== CONFIG ==========
# CLICK_SOUND = 'click.mp3'  # Place a small sound file here

# ========== HELPERS ==========
#def play_click():
 #   threading.Thread(target=playsound, args=(CLICK_SOUND,), daemon=True).start()

def distance(p1, p2):
    return math.hypot(p2[0]-p1[0], p2[1]-p1[1])


mouse_click_pos = None

def click_event(event, x, y, flags, param):
    global mouse_click_pos
    if event == cv2.EVENT_LBUTTONDOWN:
        mouse_click_pos = (x, y)
cv2.namedWindow("Gesture Calculator")
cv2.setMouseCallback("Gesture Calculator", click_event)


In [ ]:
evaluated = False
while True:
    success, img = cap.read()
    if not success or img is None:
        continue 
    img = cv2.flip(img, 1)
    h, w, _ = img.shape
    img_rgb = cv2.cvtColor(img.copy(), cv2.COLOR_BGR2RGB)
    results = hands_detector.process(img_rgb)

    # Enhanced Equation Display Box
    def draw_display_box(img, equation_text):
        x, y, w, h = 100, 60, 315, 80
        radius = 15

        # Rounded rectangle fill
        overlay = img.copy()
        fill_color = (255, 255, 255)
        border_color = (153, 204, 255)  # Light blue

        cv2.rectangle(overlay, (x + radius, y), (x + w - radius, y + h), fill_color, -1)
        cv2.rectangle(overlay, (x, y + radius), (x + w, y + h - radius), fill_color, -1)
        cv2.circle(overlay, (x + radius, y + radius), radius, fill_color, -1)
        cv2.circle(overlay, (x + w - radius, y + radius), radius, fill_color, -1)
        cv2.circle(overlay, (x + radius, y + h - radius), radius, fill_color, -1)
        cv2.circle(overlay, (x + w - radius, y + h - radius), radius, fill_color, -1)
        cv2.addWeighted(overlay, 0.92, img, 0.08, 0, img)

        # Border
        cv2.rectangle(img, (x, y), (x + w, y + h), border_color, 2)

        # Equation Text
        font = cv2.FONT_HERSHEY_PLAIN
        font_scale = 1.7
        thickness = 2
        max_width = w - 20
        text = equation_text
        while cv2.getTextSize(text, font, font_scale, thickness)[0][0] > max_width and len(text) > 1:
            text = text[1:]
        text_size = cv2.getTextSize(text, font, font_scale, thickness)[0]
        text_x = x + 10
        text_y = y + (h + text_size[1]) // 2 - 4
        cv2.putText(img, text, (text_x, text_y), font, font_scale, (0, 0, 0), thickness)

    # Call this inside your loop
    draw_display_box(img, myEquation)

    
    for btn in buttons:
        btn.hovered = False
        btn.draw(img)

        


    if results.multi_hand_landmarks:
        for handLms in results.multi_hand_landmarks:
            lmList = [(int(lm.x * w), int(lm.y * h)) for lm in handLms.landmark]
            mp_draw.draw_landmarks(img, handLms, mp_hands.HAND_CONNECTIONS)

            index = lmList[8]
            thumb = lmList[4]
            middle = lmList[12]
            ring = lmList[16]

            pinch = distance(index, thumb) < 40
            pinch_mid = distance(middle, thumb) < 40
            pinch_ring = distance(ring, thumb) < 40

            for btn in buttons:
                btn.check_hover(index[0], index[1])

            if pinch:
                for btn in buttons:
                    if btn.click(index[0], index[1]) and delayCounter == 0:
                        #play_click()
                        if btn.val == '=':
                            try:
                                myEquation = myEquation.replace('^', '**') 
                                myEquation = str(eval(myEquation))
                            except:
                                myEquation = 'Error'
                            evaluated = True
                        elif btn.val == 'C':
                            myEquation = ''
                            evaluated = False
                        elif btn.val == 'DEL':
                            if myEquation == 'Error':
                                myEquation = ''
                            else:
                                myEquation = myEquation[:-1]
                        else:
                            if evaluated and btn.val.isdigit():
                                myEquation = btn.val  # start fresh
                                
                            else:
                                myEquation += btn.val
                            evaluated = False

                        displayEquation = myEquation[-24:]
                        delayCounter = 1
            

            elif pinch_mid:
                myEquation = myEquation[:-1]
                displayEquation = myEquation[-24:]
                delayCounter = 1

            elif pinch_ring:
                myEquation = ''
                displayEquation = ''
                delayCounter = 1
    # Handle mouse clicks
    if mouse_click_pos:
        mx, my = mouse_click_pos
        for btn in buttons:
            if btn.click(mx, my):
                if btn.val == '=':
                    try:
                        myEquation = myEquation.replace('^', '**') 
                        myEquation = str(eval(myEquation))
                    except:
                        myEquation = 'Error'
                    evaluated = True
                elif btn.val == 'C':
                    myEquation = ''
                    evaluated = False
                elif btn.val == 'DEL':
                    if myEquation == 'Error':
                        myEquation = ''
                    else:
                        myEquation = myEquation[:-1]
                    evaluated = False
                else:
                    if evaluated and btn.val.isdigit():
                        myEquation = btn.val  # start fresh
                        
                    else:
                        myEquation += btn.val
                    evaluated = False
                break
        mouse_click_pos = None

    # Debounce click
    if delayCounter:
        delayCounter += 1
        if delayCounter > 10:
            delayCounter = 0


    # Display
    cv2.imshow("Gesture Calculator", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()
